In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

import time
import pandas as pd

In [19]:
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome("chrome_driver\chromedriver.exe", options=options)

# 메인 홈페이지로 이동
driver.get("https://www.koreabaseball.com/")

# 기록 페이지로 이동
css_selector = "#lnb > li:nth-child(3)"
elem = driver.find_element(By.CSS_SELECTOR, css_selector)
elem.click()

# 기록실 페이지로 이동
css_selector = "#lnb02"
elem = driver.find_element(By.CSS_SELECTOR, css_selector)
elem.click()

C:\Users\Luna\AppData\Local\Temp\ipykernel_18920\1968520446.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chrome_driver\chromedriver.exe", options=options)


In [20]:
options = ['hitter', 'pitcher', 'defense', 'runner']
record_tab_xpath = '//*[@id="contents"]/div[2]/div[2]/ul/*'
for i in range(0, 4):
    status = options[i]
    
    record_tab = driver.find_elements(By.XPATH, record_tab_xpath)
    record_tab[i].click()
    
    time.sleep(3)

    select = Select(driver.find_element(By.NAME, "ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$ddlTeam$ddlTeam"))
    select.select_by_value(value='OB')

    time.sleep(3)

    table_value = driver.find_element(By.CLASS_NAME, 'tData01').text.split('\n')
    table_head = table_value.pop(0).split(' ')
    
    # 페이지 이동
    pages = driver.find_element(By.CLASS_NAME, 'paging').text.split(' ')
    if len(pages) == 1:
        pass
    else:
        for page in range(1, len(pages)):
            id_selector = 'cphContents_cphContents_cphContents_ucPager_btnNo' + str(page+1)
            elem = driver.find_element(By.ID, id_selector)
            elem.click()

            time.sleep(3)

            temp = driver.find_element(By.CLASS_NAME, 'tData01').text.split('\n')
            del temp[0]
            table_value.extend(temp)

    record_values = []

    for item in table_value:
        record_values.append(item.split(' '))
    
    if status == 'pitcher':
        for item in record_values:
            if len(item) == len(table_head):
                pass
            else:
                temp1 = item.pop(10)
                temp2 = item.pop(10)
                res_temp = temp1 + ' ' + temp2
                item.insert(10, res_temp)

    if status == 'defense':
        for item in record_values:
            if len(item) == len(table_head):
                pass
            else:
                temp1 = item.pop(6)
                temp2 = item.pop(6)
                res_temp = temp1 + ' ' + temp2
                item.insert(6, res_temp)
    
    globals()[f'{status}_dataframe'] = pd.DataFrame(columns=table_head)
    
    for j in range(0, len(record_values)):
        globals()[f'{status}_dataframe'].loc[j] = record_values[j]

    time.sleep(2)

with pd.ExcelWriter('test.xlsx') as writer:
    for status in options:
        globals()[f'{status}_dataframe'].to_excel(writer, sheet_name=status, index=False)

Empty DataFrame
Columns: [순위, 선수명, 팀명, AVG, G, PA, AB, R, H, 2B, 3B, HR, TB, RBI, SAC, SF]
Index: []
Empty DataFrame
Columns: [순위, 선수명, 팀명, ERA, G, W, L, SV, HLD, WPCT, IP, H, HR, BB, HBP, SO, R, ER, WHIP]
Index: []
Empty DataFrame
Columns: [순위, 선수명, 팀명, POS, G, GS, IP, E, PKO, PO, A, DP, FPCT, PB, SB, CS, CS%]
Index: []
Empty DataFrame
Columns: [순위, 선수명, 팀명, G, SBA, SB, CS, SB%, OOB, PKO]
Index: []
